# Prepare data to build a 3D model
## Load dataset, prepare the geo_model and visualize data
Kristiaan JOSEPH and Olivier KAUFMANN, 2020

In [1]:
%matplotlib inline
import geopandas as gpd
import gempy as gp
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
gp.__version__

'2.2.9'

In [3]:
gdf = gpd.read_file('../../../../data/Parties exploitées - Caillette.gpkg', layer = 'Parties exploitées - Caillette')

In [4]:
def polygon_gdf_to_points_gdf(gdf, columns=[]):
    geo_interface = {"type": "FeatureCollection"}
    features = []
    bbox = (gdf.iloc[:]['geometry'].bounds.minx.min(), gdf.iloc[:]['geometry'].bounds.maxx.max(), gdf.iloc[:]['geometry'].bounds.miny.min(), gdf.iloc[:]['geometry'].bounds.maxy.max())
    geo_interface["bbox"] = bbox
    point_id = 0
    for idx, row in gdf.iterrows():
        polygon = row['geometry']    
        vertices = np.array(polygon.exterior.coords[:])
        for j in range(vertices.shape[1]):
            properties = {p : row[p] for p in columns}
            pt = {"id": f'{point_id}', "type": "Feature", "properties": properties,
                 "geometry": {"type": "Point", "coordinates": tuple(vertices[j,:])},
                 }
            features.append(pt)
            point_id += 1
    geo_interface["features"] = features

    return gpd.GeoDataFrame.from_features(geo_interface)

In [5]:
surface_points_gdf = polygon_gdf_to_points_gdf(gdf, columns=['Couche ', 'Année'])

In [6]:
surface_points_gdf['X'] = surface_points_gdf['geometry'].apply(lambda geom: geom.x)
surface_points_gdf['Y'] = surface_points_gdf['geometry'].apply(lambda geom: geom.y)
surface_points_gdf['Z'] = surface_points_gdf['geometry'].apply(lambda geom: geom.z)

In [7]:
surface_points_gdf.drop('Année', axis=1, inplace=True)
surface_points_gdf.rename(columns={'Couche ': 'formation'}, inplace=True)

In [8]:
surface_points_df = pd.DataFrame(surface_points_gdf[['X', 'Y', 'Z', 'formation']])

In [9]:
surface_points_df.to_csv('./tmp_files/surface_points.csv')

In [4]:
data_path= './tmp_file/'
orientations_file = data_path + 'orientations.csv'
surface_points_file = data_path + 'surface_points.csv'
strat_pile_file = data_path + 'strat_pile.csv'
model_extent = [0.,100.,0.,100.,0.,60.]
model_resolution = [50, 50, 30]

In [5]:
# Importing the data from CSV-files and setting extent and resolution
geo_data = gp.create_data('test', model_extent, model_resolution, 
      path_o = orientations_file,
      path_i = surface_points_file, default_values=True); 

Active grids: ['regular']


In [6]:
min_X = min(geo_data.surface_points.df['X'].min(), geo_data.orientations.df['X'].min())
max_X = max(geo_data.surface_points.df['X'].max(), geo_data.orientations.df['X'].max())
min_Y = min(geo_data.surface_points.df['Y'].min(), geo_data.orientations.df['Y'].min())
max_Y = max(geo_data.surface_points.df['Y'].max(), geo_data.orientations.df['Y'].max())
min_Z = min(geo_data.surface_points.df['Z'].min(), geo_data.orientations.df['Z'].min())
max_Z = max(geo_data.surface_points.df['Z'].max(), geo_data.orientations.df['Z'].max())
print('[', min_X+10, max_X+10, min_Y+10, max_Y+10, min_Z, max_Z+10,']')

[ 20 60 30 70 10 50 ]


In [7]:
geo_data

test  2021-10-23 10:55

random topography

In [8]:
geo_data.set_topography(d_z=np.array([20., 50.]), fd=1.8)

Active grids: ['regular' 'topography']


Grid Object. Values: 
array([[  1.        ,   1.        ,   1.        ],
       [  1.        ,   1.        ,   3.        ],
       [  1.        ,   1.        ,   5.        ],
       ...,
       [100.        ,  95.91836735,  34.01386585],
       [100.        ,  97.95918367,  34.91944182],
       [100.        , 100.        ,  35.84198637]])

In [9]:
geo_data._grid.topography.save('topo.npy')

simple flat topography

In [10]:
topo = np.load('topo.npy')
topo.shape

(50, 50, 3)

In [11]:
for i in range(model_resolution[0]):
    for j in range(model_resolution[1]):
        topo[i,j,2] = 40.-i/5.
np.save('topo.npy', topo)

In [12]:
geo_data.set_topography(source='numpy', array=topo)

Active grids: ['regular' 'topography']


Grid Object. Values: 
array([[  1.        ,   1.        ,   1.        ],
       [  1.        ,   1.        ,   3.        ],
       [  1.        ,   1.        ,   5.        ],
       ...,
       [100.        ,  95.91836735,  30.2       ],
       [100.        ,  97.95918367,  30.2       ],
       [100.        , 100.        ,  30.2       ]])

In [13]:
geo_data.series

,order_series,BottomRelation,isActive,isFault,isFinite
Default series,1,Erosion,True,False,False
Basement,2,Erosion,False,False,False


In [14]:
gp.map_series_to_surfaces(geo_data, {"Strat_Series": ('Default series')})

,surface,series,order_surfaces,color,id
0,A,Default series,1,#015482,1
1,B,Default series,2,#9f0052,2
2,basement,Basement,1,#ffbe00,3


In [15]:
geo_data.surfaces

,surface,series,order_surfaces,color,id
0,A,Default series,1,#015482,1
1,B,Default series,2,#9f0052,2
2,basement,Basement,1,#ffbe00,3


### Read the full strat_pile, filter out surfaces that are not within the geomodel surfaces and reorder surfaces in "Default series" according to the order in the strat_pile_file

In [16]:
strat_pile = pd.read_csv(strat_pile_file, header=0, names=['surface', 'order']) # if no headers are present in strat_pile_file, put header=None

In [17]:
strat_pile = strat_pile.merge(geo_data.surfaces.df, how='left', left_on='surface', right_on='surface').query('(series==series) and (series=="Default series")').sort_values('order')[['surface', 'order']]

In [18]:
strat_pile_list = strat_pile['surface'].tolist()

In [19]:
current_order = geo_data.surfaces.df.query('series=="Default series"')['surface'].to_dict()

In [20]:
current_order = {v:k for k, v in current_order.items()}

Create the new order from the strat_pile_list

In [21]:
new_order = {}
for k in range(len(strat_pile_list)):
    new_order.update({strat_pile_list[k] : k+1})

Reorder surfaces

In [22]:
geo_data.surfaces

,surface,series,order_surfaces,color,id
0,A,Default series,1,#015482,1
1,B,Default series,2,#9f0052,2
2,basement,Basement,1,#ffbe00,3


In [23]:
geo_data.series

,order_series,BottomRelation,isActive,isFault,isFinite
Default series,1,Erosion,True,False,False
Basement,2,Erosion,False,False,False


### Check data in 3D

In [24]:
gp.plot.plot_3d(geo_data, image=False, show_topography=True, plotter_type='background', notebook=True);

In [25]:
geo_data

test  2021-10-23 10:55